## Traditional machine learning
Feature(s): A,AB,ABC,ABCD,B
- These columns are all list and increasinlgy add the values of a 'property class'
- group A are all propeties related to the pore 
- group B are all propeties related to the material
- group C are all propeties related to the composition  
- group D are all propeties related to the ...  
- try them all to make a comparision

target: Uptake_bool,Diffusion_bool
- two binary classifications


In [1]:
import sys
import os
from sklearn.model_selection import cross_val_score
from sklearn.base import clone
from loguru import logger

logger.remove()
logger.add(sys.stderr, level="WARNING")
# Append the parent directory of your package to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))

Read the dataset from the zip file

In [2]:
import pandas as pd
import numpy as np
import zipfile

path_to_dataset = 'helium_all.csv'
csv_filename = 'helium_all.csv'

# Open the file, Correct the encoding and sep if necessary
if path_to_dataset.endswith('.zip'):
    with zipfile.ZipFile(path_to_dataset, 'r') as z:
        # Open the CSV file within the ZIP file
        with z.open(csv_filename) as f:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(f, sep=',', on_bad_lines='warn', index_col = 0)
else:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(path_to_dataset, sep=',', on_bad_lines='warn', index_col = 0)



In [3]:
import ast

df['A'] = df['A'].apply(ast.literal_eval)
df['AB'] = df['AB'].apply(ast.literal_eval)
df['ABC'] = df['ABC'].apply(ast.literal_eval)
df['ABCD'] = df['ABCD'].apply(ast.literal_eval)
df['B'] = df['B'].apply(ast.literal_eval)


In [4]:
from MLPipeline import MLmodel, BinTheTarget

Target = ['Uptake_bool','Diffusion_bool']
Features = ['A','AB','ABC','ABCD','B']
Feature_types = ['numerical']* sum([len(df[f][0]) for f in Features])
input = df

/home/lsmo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def objectiveRandomForestClassifier(trial, model_instance):
    """
    Objective function for Optuna to minimize.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_categorical('max_depth', [None, 10, 20, 30, 40]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 6),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }

    # Clone the model to ensure a fresh instance each trial
    model_clone = clone(model_instance.model)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()


def objectiveXGBClassifier(trial, model_instance):
    """
    Objective function for Optuna to minimize for XGBClassifier.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),  # L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),  # L2 regularization
    }

    # Clone the model to ensure a fresh instance each trial
    model_clone = clone(model_instance.model)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()


In [6]:
MODEL_NAMES = ['RandomForestClassifier', 'XGBClassifier']
TARGETS = [[i] for i in Target ]
TRAIN_SIZES = [25, 50, 100, 200]

result = []

for feature in Features:

    # Split the column feature into multiple columns
    input_df = df.copy()
    df_split = pd.DataFrame(df[feature].tolist(), columns=[f'{str(feature)}{i}' for i in range(len(df[feature][0]))])
    input_df = input_df.drop(columns=[feature])
    input_features = [f'{str(feature)}{i}' for i in range(len(df[feature][0]))]
    input_feature_types = ['numerical']* len(input_features)
    input_df = input_df.join(df_split)


    for seed in [1, 2, 3]:
        for model_name in MODEL_NAMES:
            for target in TARGETS:
                for train_size in TRAIN_SIZES:
                    print(f'RUN: Model:{model_name} / Target:{target} / Train size:{train_size} / Seed:{seed}')
                    
                    if model_name == 'RandomForestClassifier':
                        objective = objectiveRandomForestClassifier
                    elif model_name == 'XGBClassifier':
                        objective = objectiveXGBClassifier

                    model = MLmodel(modelType=model_name, 
                    df=input_df,
                    randomSeed=seed,
                    train_count = train_size,
                    test_count = 50, 
                    target=target, 
                    features=input_features, 
                    hyperparameter_tuning=True,
                    feature_types=input_feature_types,
                    optimization_method='optuna', 
                    optimization_trials=10,
                    objective=lambda trial: objective(trial, model)
                    )

                    model.train()
                    eval, summary = model.evaluate()
                    result.append({**summary, **eval, 'seed':seed, 'features':input_features})



[I 2024-09-17 16:10:44,876] A new study created in memory with name: no-name-db2e3a41-0a7e-4c07-8101-bfc81a184d40


RUN: Model:RandomForestClassifier / Target:['Uptake_bool'] / Train size:25 / Seed:1


[I 2024-09-17 16:10:45,244] Trial 0 finished with value: 0.64 and parameters: {'n_estimators': 96, 'max_depth': 40, 'min_samples_split': 7, 'min_samples_leaf': 3, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 0.64.
[I 2024-09-17 16:10:46,058] Trial 1 finished with value: 0.7200000000000001 and parameters: {'n_estimators': 196, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 1 with value: 0.7200000000000001.
[I 2024-09-17 16:10:46,325] Trial 2 finished with value: 0.6 and parameters: {'n_estimators': 66, 'max_depth': 30, 'min_samples_split': 11, 'min_samples_leaf': 6, 'max_features': 'log2', 'bootstrap': False}. Best is trial 1 with value: 0.7200000000000001.
[I 2024-09-17 16:10:47,754] Trial 3 finished with value: 0.7200000000000001 and parameters: {'n_estimators': 234, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': True}. Best is t

RUN: Model:RandomForestClassifier / Target:['Uptake_bool'] / Train size:50 / Seed:1


[I 2024-09-17 16:10:52,729] Trial 0 finished with value: 0.6799999999999999 and parameters: {'n_estimators': 172, 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 0.6799999999999999.
[I 2024-09-17 16:10:53,372] Trial 1 finished with value: 0.6799999999999999 and parameters: {'n_estimators': 115, 'max_depth': 40, 'min_samples_split': 7, 'min_samples_leaf': 3, 'max_features': 'log2', 'bootstrap': True}. Best is trial 0 with value: 0.6799999999999999.
[I 2024-09-17 16:10:53,914] Trial 2 finished with value: 0.7 and parameters: {'n_estimators': 138, 'max_depth': 30, 'min_samples_split': 7, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 2 with value: 0.7.
[I 2024-09-17 16:10:55,106] Trial 3 finished with value: 0.66 and parameters: {'n_estimators': 284, 'max_depth': 20, 'min_samples_split': 13, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': False}. Best is 

KeyboardInterrupt: 

Some information about the dataset such as the splitting, wrong smiles, total number of samples.

In [ ]:
df = pd.DataFrame(result)
display(df)
df.to_csv('GasUptakeDiffusion_MOFS_He_traditional_ml.csv')

In [ ]:
import matplotlib.pyplot as plt

METRICS = ['accuracy','f1_micro','f1_macro', 'kappa']
fig, ax = plt.subplots(len(METRICS), 1, sharex=True, layout = 'constrained')
ax = ax.flatten()

for i, metric in enumerate(METRICS):
    ax[i].plot(df['train_size'], df[metric], marker = 'o')
    ax[i].set_title(metric)
    ax[i].set_ylim(0,1)

fig.suptitle('Random Forrest & XGB - GasUptakeDiffusion_MOFS_He')